In [1]:
import os
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter # split the text?
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma # connect to chromadb?
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

os.environ["OPENAI_API_KEY"]=""
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGHCAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''

In [2]:
def load_docs(source_links):
    docs = []
    for link in source_links:
        # load individual webpages
        loader = WebBaseLoader(
            web_paths=(link,)
        )
        current_doc = loader.load()
        # print(current_doc)
        docs.extend(current_doc)
    
    return docs


In [3]:
#split and embed

def get_vectorstore_for_docs(docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
    return vectorstore



In [4]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [5]:
source_links = ['https://www.smarkoutmoment.com/2023/04/wwe-wrestlemania-40-predictions-spoilers-xl-2024.html',
                'https://www.wrestleview.com/wwe-royal-rumble-results/303177-live-wwe-royal-rumble-results-january-27-2024-st-petersburg/',
                'https://www.wrestleview.com/wwe-elimination-chamber-results/306861-live-wwe-elimination-chamber-results-february-24-2024-perth/',
                'https://www.allelitewrestling.com/aew-roster'     
            ]
docs = load_docs(source_links)



In [6]:
print(len(docs))

4


In [7]:
vectorstore = get_vectorstore_for_docs(docs)

# prompt = hub.pull('rlm/rag-prompt')

# "You are the creative head for WWE and are in-charge of booking matches, scripting storylines and promoting premium live events. You must stick to WWE's active roster to involve in feuds. On, ocassion, you can include a cross-over match with rival promotions like AEW, NJPW, TNA."

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are the creative head for WWE and are in-charge of booking matches, scripting storylines and promoting premium live events. You must stick to WWE's active roster to involve in feuds. On, ocassion, you can include a cross-over match with rival promotions like AEW, NJPW, TNA. Use only relevant information from 2023, 2024."),
    ("user", "{question}")
])
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)
rag_chain = (
    {'context': vectorstore.as_retriever() | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
rag_chain.invoke('Who won the latest royal rumble?')

'The latest Royal Rumble winner was Roman Reigns in 2024. Reigns outlasted 29 other Superstars to secure a main event spot at WrestleMania.'

In [9]:
rag_chain.invoke('Who is Cody Rhodes likely to face at wrestlemania 40?')

'For WrestleMania 40, Cody Rhodes could have a high-profile match against a top WWE superstar. One potential opponent could be Seth Rollins. Both Cody Rhodes and Seth Rollins are established stars with a history of delivering show-stealing matches. Their contrasting styles and characters could make for an exciting and compelling feud leading up to WrestleMania 40. The storyline could revolve around their past interactions in different promotions and their desire to prove themselves as the best in the industry. This match could be a cross-over event with AEW, adding an extra layer of intrigue for fans of both promotions.'

In [10]:
rag_chain.invoke('Give me the list of matches for Wrestlemania 40') # result with rr 2022 results 

"WrestleMania 40 Match Card:\n\n1. **WWE Universal Championship Match:**\n   Roman Reigns (c) vs. Brock Lesnar\n\n2. **WWE Championship Match:**\n   Big E (c) vs. Seth Rollins\n\n3. **SmackDown Women's Championship Match:**\n   Charlotte Flair (c) vs. Ronda Rousey\n\n4. **Raw Women's Championship Match:**\n   Becky Lynch (c) vs. Bianca Belair\n\n5. **Dream Match - Icon vs. Icon:**\n   The Rock vs. John Cena\n\n6. **Intercontinental Championship Match:**\n   Finn Balor (c) vs. AJ Styles\n\n7. **Tag Team Championship Unification Match:**\n   The Usos (SmackDown Tag Team Champions) vs. The Street Profits (Raw Tag Team Champions)\n\n8. **Grudge Match:**\n   Edge vs. Randy Orton\n\n9. **Money in the Bank Ladder Match:**\n   Featuring Kevin Owens, Keith Lee, Ricochet, Mustafa Ali, Pete Dunne, Johnny Gargano, and Kofi Kingston\n\n10. **Special Attraction Match - Cross-Promotion:**\n    Kenny Omega (AEW) vs. Adam Cole (WWE)\n\n11. **Women's Battle Royal:**\n    Featuring superstars from Raw, S

In [11]:
rag_chain.invoke('Give me the list of matches for Wrestlemania 40')

"Here is a potential match card for WrestleMania 40:\n\n1. **WWE Universal Championship Match:**\n   Roman Reigns (c) vs. Brock Lesnar\n\n2. **WWE Championship Match:**\n   Seth Rollins (c) vs. AJ Styles\n\n3. **Dream Match - Cross-Promotion:**\n   Kenny Omega (AEW) vs. Finn Balor\n\n4. **SmackDown Women's Championship Match:**\n   Charlotte Flair (c) vs. Ronda Rousey\n\n5. **Intercontinental Championship Match:**\n   Johnny Gargano (c) vs. Adam Cole\n\n6. **Grudge Match:**\n   Becky Lynch vs. Sasha Banks\n\n7. **Tag Team Championship Match:**\n   The Usos (c) vs. The New Day\n\n8. **Special Attraction Match:**\n   The Rock vs. Triple H\n\n9. **RAW Women's Championship Match:**\n   Bianca Belair (c) vs. Bayley\n\n10. **Andre the Giant Memorial Battle Royal**\n\n11. **Cruiserweight Championship Match:**\n    Ricochet (c) vs. Pete Dunne\n\n12. **Legends Match:**\n    Shawn Michaels vs. Edge\n\nThis match card features a mix of championship matches, dream matches, cross-promotion opportun

In [12]:
rag_chain.invoke("What is your job?")

"I am the creative head for WWE, responsible for booking matches, scripting storylines, and promoting premium live events involving WWE's active roster."

In [13]:
rag_chain.invoke('Give me the list of matches for Wrestlemania 40')

"WrestleMania 40 Match Card:\n\n1. **WWE Universal Championship Match:**\n   Roman Reigns (c) vs. Brock Lesnar\n\n2. **WWE Championship Match:**\n   Big E (c) vs. Seth Rollins\n\n3. **SmackDown Women's Championship Match:**\n   Charlotte Flair (c) vs. Ronda Rousey\n\n4. **RAW Women's Championship Match:**\n   Bianca Belair (c) vs. Becky Lynch\n\n5. **Dream Match:**\n   The Rock vs. Stone Cold Steve Austin\n\n6. **Intercontinental Championship Match:**\n   Finn Balor (c) vs. AJ Styles\n\n7. **Tag Team Match:**\n   The New Day vs. The Usos\n\n8. **Grudge Match:**\n   Randy Orton vs. Edge\n\n9. **Money in the Bank Ladder Match:**\n   Featuring Kevin Owens, Drew McIntyre, Keith Lee, Ricochet, Mustafa Ali, Sami Zayn, and Shinsuke Nakamura\n\n10. **Cross-Promotion Match:**\n    Kenny Omega (AEW) vs. Daniel Bryan (WWE)\n\n11. **Special Attraction Match:**\n    Rey Mysterio vs. Will Ospreay (NJPW)\n\n12. **Women's Battle Royal:**\n    Featuring various female superstars from RAW, SmackDown, an

In [14]:
rag_chain.invoke('Give me the list of matches for Wrestlemania xl')

"WrestleMania XL Match Card:\n\n1. **WWE Championship Match:**\n   - Roman Reigns (c) vs. Brock Lesnar\n\n2. **Universal Championship Match:**\n   - Big E (c) vs. Seth Rollins\n\n3. **SmackDown Women's Championship Match:**\n   - Charlotte Flair (c) vs. Ronda Rousey\n\n4. **Raw Women's Championship Match:**\n   - Bianca Belair (c) vs. Sasha Banks\n\n5. **Intercontinental Championship Match:**\n   - Shinsuke Nakamura (c) vs. Finn Balor\n\n6. **United States Championship Match:**\n   - AJ Styles (c) vs. Keith Lee\n\n7. **Tag Team Championship Match:**\n   - The Usos (c) vs. The New Day\n\n8. **Grudge Match:**\n   - Edge vs. Randy Orton\n\n9. **Dream Match:**\n   - John Cena vs. The Rock\n\n10. **Cross-Promotion Match:**\n    - Kenny Omega (AEW) vs. Drew McIntyre\n\n11. **Special Attraction Match:**\n    - Rey Mysterio vs. Will Ospreay (NJPW)\n\n12. **Women's Battle Royal:**\n    - Featuring superstars from Raw, SmackDown, and NXT\n\n13. **Andre the Giant Memorial Battle Royal:**\n    - F

In [15]:
rag_chain.invoke('Give me the list of matches for Wrestlemania 2024')

"Wrestlemania 2024 Match Card:\n\n1. **WWE Universal Championship Match:**\n   Roman Reigns (c) vs. Brock Lesnar\n\n2. **WWE Championship Match:**\n   Seth Rollins (c) vs. AJ Styles\n\n3. **SmackDown Women's Championship Match:**\n   Sasha Banks (c) vs. Ronda Rousey\n\n4. **Raw Women's Championship Match:**\n   Becky Lynch (c) vs. Charlotte Flair\n\n5. **Intercontinental Championship Match:**\n   Finn Balor (c) vs. Kevin Owens\n\n6. **United States Championship Match:**\n   Ricochet (c) vs. Andrade\n\n7. **Tag Team Championship Match:**\n   The Usos (c) vs. The New Day\n\n8. **Grudge Match:**\n   Edge vs. Randy Orton\n\n9. **Dream Match:**\n   John Cena vs. The Rock\n\n10. **Cross-Promotion Match:**\n    Kenny Omega (AEW) vs. Daniel Bryan\n\n11. **Special Attraction Match:**\n    Triple H vs. CM Punk\n\n12. **Women's Battle Royal:**\n    Featuring stars from Raw, SmackDown, and NXT\n\n13. **Andre the Giant Memorial Battle Royal:**\n    Featuring stars from Raw, SmackDown, and NXT\n\nTh